## Quiz #0502

### "Human Activity Recognition" 

#### Answer the following questions by providing Python code:
#### Objectives:
- Carry out the EDA.
- Carry out the data pre-processing.
- Optimize and test a predictive model of your choice.

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import metrics, preprocessing
warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

#### Read in data:
The explanation on the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones).

In [28]:
# Go to the directory where the data file is located. 
# os.chdir(r'~~')                # Please, replace the path with your own. 

In [29]:
df = pd.read_csv('data_human activity recognition.csv', header='infer')

In [30]:
df.shape

(19622, 160)

In [31]:
df.columns

Index(['Unnamed: 0', 'user_name', 'raw_timestamp_part_1',
       'raw_timestamp_part_2', 'cvtd_timestamp', 'new_window', 'num_window',
       'roll_belt', 'pitch_belt', 'yaw_belt',
       ...
       'gyros_forearm_x', 'gyros_forearm_y', 'gyros_forearm_z',
       'accel_forearm_x', 'accel_forearm_y', 'accel_forearm_z',
       'magnet_forearm_x', 'magnet_forearm_y', 'magnet_forearm_z', 'classe'],
      dtype='object', length=160)

1). Carry out the EDA. Check for the missing values. HINT: The response variable is 'classe'.

In [32]:
df.describe()

,Unnamed: 0,raw_timestamp_part_1,raw_timestamp_part_2,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,max_roll_belt,max_picth_belt,min_roll_belt,min_pitch_belt,amplitude_roll_belt,amplitude_pitch_belt,var_total_accel_belt,avg_roll_belt,stddev_roll_belt,var_roll_belt,avg_pitch_belt,stddev_pitch_belt,var_pitch_belt,avg_yaw_belt,stddev_yaw_belt,var_yaw_belt,gyros_belt_x,gyros_belt_y,gyros_belt_z,accel_belt_x,accel_belt_y,accel_belt_z,magnet_belt_x,magnet_belt_y,magnet_belt_z,roll_arm,pitch_arm,yaw_arm,total_accel_arm,var_accel_arm,avg_roll_arm,stddev_roll_arm,...,stddev_yaw_dumbbell,var_yaw_dumbbell,gyros_dumbbell_x,gyros_dumbbell_y,gyros_dumbbell_z,accel_dumbbell_x,accel_dumbbell_y,accel_dumbbell_z,magnet_dumbbell_x,magnet_dumbbell_y,magnet_dumbbell_z,roll_forearm,pitch_forearm,yaw_forearm,max_roll_forearm,max_picth_forearm,min_roll_forearm,min_pitch_forearm,amplitude_roll_forearm,amplitude_pitch_forearm,total_accel_forearm,var_accel_forearm,avg_roll_forearm,stddev_roll_forearm,var_roll_forearm,avg_pitch_forearm,stddev_pitch_forearm,var_pitch_forearm,avg_yaw_forearm,stddev_yaw_forearm,var_yaw_forearm,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z
count,19622.000000,1.962200e+04,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,406.000000,406.000000,406.000000,...,406.000000,406.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,19622.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000
mean,9811.500000,1.322827e+09,500656.144277,430.640047,64.407197,0.305283,-11.205061,11.312608,-6.667241,12.923645,-10.436453,10.756158,3.768941,2.167488,0.926191,68.055576,1.337113,7.699186,0.519510,0.602692,0.765692,-8.830781,1.341249,107.486512,-0.005592,0.039588,-0.130538,-5.594690,30.154877,-72.593619,55.600856,593.676995,-345.480838,17.830183,-4.611925,-0.618751,25.508562,53.235180,12.677667,11.200904,...,16.646833,589.839334,0.161084,0.046056,-0.128989,-28.615686,52.633218,-38.322393,-328.480787,220.965192,46.053695,33.826536,10.705989,19.207633,24.486453,81.486700,-0.167488,-57.571921,24.653374,139.058350,34.715320,33.501502,33.165276,41.985994,5274.095349,11.795012,7.977324,139.593267,17.998348,44.854413,4639.849068,0.157951,0.075175,0.151245,-61.651819,163.655896,-55.291917,-312.575884,380.116445,393.613745
std,5664.527827,2.049277e+05,288222.879958,247.909554,62.750255,22.351242,95.193926,7.742309,94.594252,8.005960,93.616774,7.470908,25.258542,2.361529,2.222549,63.136137,2.435055,23.161413,22.405946,0.639322,1.763079,93.475194,10.292995,1655.520026,0.207329,0.078236,0.241321,29.644785,28.577835,100.446740,64.179286,35.680855,65.209549,72.744054,30.681952,71.363384,10.522272,53.979570,68.576727,17.103335,...,17.705775,1244.593568,1.508553,0.609990,2.286599,67.316762,80.750641,109.468471,339.724224,326.865984,139.963761,108.035861,28.145534,103.219156,31.043450,95.537755,22.590453,110.743609,25.884850,147.864943,10.055956,33.951744,79.520306,59.329094,9177.177020,24.833838,8.726002,266.494879,77.559125,51.326595,7284.972361,0.648618,3.100725,1.754483,180.593687,200.130082,138.396947,346.958482,509.373742,369.268747
min,1.000000,1.322490e+09,294.000000,1.000000,-28.900000,-55.800000,-180.000000,0.000000,-94.30000

In [33]:
df.classe.value_counts()

A    5580
B    3797
E    3607
C    3422
D    3216
Name: classe, dtype: int64

2). Get rid of the columns that have more than 97% missing values.

In [34]:
l = df.isnull().sum() > 1638

In [35]:
yourdf = df.dropna(thresh=1638, axis=1)

In [36]:
yourdf.classe.dropna()

0        A
1        A
2        A
3        A
4        A
        ..
19617    E
19618    E
19619    E
19620    E
19621    E
Name: classe, Length: 19622, dtype: object

In [37]:
yourdf.shape

(19622, 60)

In [38]:
yourdf.describe()

,Unnamed: 0,raw_timestamp_part_1,raw_timestamp_part_2,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,gyros_belt_x,gyros_belt_y,gyros_belt_z,accel_belt_x,accel_belt_y,accel_belt_z,magnet_belt_x,magnet_belt_y,magnet_belt_z,roll_arm,pitch_arm,yaw_arm,total_accel_arm,gyros_arm_x,gyros_arm_y,gyros_arm_z,accel_arm_x,accel_arm_y,accel_arm_z,magnet_arm_x,magnet_arm_y,magnet_arm_z,roll_dumbbell,pitch_dumbbell,yaw_dumbbell,total_accel_dumbbell,gyros_dumbbell_x,gyros_dumbbell_y,gyros_dumbbell_z,accel_dumbbell_x,accel_dumbbell_y,accel_dumbbell_z,magnet_dumbbell_x,magnet_dumbbell_y,magnet_dumbbell_z,roll_forearm,pitch_forearm,yaw_forearm,total_accel_forearm,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z
count,19622.000000,1.962200e+04,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.00000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000,19622.000000
mean,9811.500000,1.322827e+09,500656.144277,430.640047,64.407197,0.305283,-11.205061,11.312608,-0.005592,0.039588,-0.130538,-5.594690,30.154877,-72.593619,55.600856,593.676995,-345.480838,17.830183,-4.611925,-0.618751,25.508562,0.042772,-0.257081,0.269480,-60.240190,32.597085,-71.245082,191.722964,156.611304,306.49098,23.841512,-10.779420,1.674361,13.718377,0.161084,0.046056,-0.128989,-28.615686,52.633218,-38.322393,-328.480787,220.965192,46.053695,33.826536,10.705989,19.207633,34.715320,0.157951,0.075175,0.151245,-61.651819,163.655896,-55.291917,-312.575884,380.116445,393.613745
std,5664.527827,2.049277e+05,288222.879958,247.909554,62.750255,22.351242,95.193926,7.742309,0.207329,0.078236,0.241321,29.644785,28.577835,100.446740,64.179286,35.680855,65.209549,72.744054,30.681952,71.363384,10.522272,1.993597,0.851391,0.553169,182.044905,109.866023,134.652852,443.643325,201.910485,326.61929,69.931720,36.993513,82.517192,10.233435,1.508553,0.609990,2.286599,67.316762,80.750641,109.468471,339.724224,326.865984,139.963761,108.035861,28.145534,103.219156,10.055956,0.648618,3.100725,1.754483,180.593687,200.130082,138.396947,346.958482,509.373742,369.268747
min,1.000000,1.322490e+09,294.000000,1.000000,-28.900000,-55.800000,-180.000000,0.000000,-1.040000,-0.640000,-1.460000,-120.000000,-69.000000,-275.000000,-52.000000,354.000000,-623.000000,-180.000000,-88.800000,-180.000000,1.000000,-6.370000,-3.440000,-2.330000,-404.000000,-318.000000,-636.000000,-584.000000,-392.000000,-597.00000,-153.713729,-149.593648,-150.871154,0.000000,-204.000000,-2.100000,-2.380000,-419.000000,-189.000000,-334.000000,-643.000000,-3600.000000,-262.000000,-180.000000,-72.500000,-180.000000,0.000000,-22.000000,-7.020000,-8.090000,-498.000000,-632.000000,-446.000000,-1280.000000,-896.000000,-973.000000
25%,4906.250000,1.322673e+09,252912.250000,222.000000,1.100000,1.760000,-88.300000,3.000000,-0.030000,0.000000,-0.200000,-21.000000,3.000000,-162.000000,9.000000,581.000000,-375.000000,-31.775000,-25.900000,-43.100000,17.000000,-1.330000,-0.800000,-0.070000,-242.000000,-54.000000,-143.000000,-300.000000,-9.000000,131.25000,-18.493424,-40.886884,-77.644206,4.000000,-0.030000,-0.140000,-0.310000,-50.000000,-8.000000,-142.000000,-535.000000,231.000000,-45.000000,-0.737500,0.000000,-68.600000,29.000000,-0.220000,-1.460000,-0.180000,-178.000000,57.000000,-182.000000,-616.000000,2.000000,191.000000
50%,9811.500000,1.322833e+09,496380.000000,424.000000,113.0

In [39]:
yourdf

,Unnamed: 0,user_name,raw_timestamp_part_1,raw_timestamp_part_2,cvtd_timestamp,new_window,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,gyros_belt_x,gyros_belt_y,gyros_belt_z,accel_belt_x,accel_belt_y,accel_belt_z,magnet_belt_x,magnet_belt_y,magnet_belt_z,roll_arm,pitch_arm,yaw_arm,total_accel_arm,gyros_arm_x,gyros_arm_y,gyros_arm_z,accel_arm_x,accel_arm_y,accel_arm_z,magnet_arm_x,magnet_arm_y,magnet_arm_z,roll_dumbbell,pitch_dumbbell,yaw_dumbbell,total_accel_dumbbell,gyros_dumbbell_x,gyros_dumbbell_y,gyros_dumbbell_z,accel_dumbbell_x,accel_dumbbell_y,accel_dumbbell_z,magnet_dumbbell_x,magnet_dumbbell_y,magnet_dumbbell_z,roll_forearm,pitch_forearm,yaw_forearm,total_accel_forearm,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
0,1,carlitos,1323084231,788290,05/12/2011 11:23,no,11,1.41,8.07,-94.4,3,0.00,0.00,-0.02,-21,4,22,-3,599,-313,-128.0,22.5,-161.0,34,0.00,0.00,-0.02,-288,109,-123,-368,337,516,13.052175,-70.494004,-84.873939,37,0.00,-0.02,0.00,-234,47,-271,-559,293,-65.0,28.4,-63.9,-153.0,36,0.03,0.00,-0.02,192,203,-215,-17,654.0,476.0,A
1,2,carlitos,1323084231,808298,05/12/2011 11:23,no,11,1.41,8.07,-94.4,3,0.02,0.00,-0.02,-22,4,22,-7,608,-311,-128.0,22.5,-161.0,34,0.02,-0.02,-0.02,-290,110,-125,-369,337,513,13.130740,-70.637505,-84.710647,37,0.00,-0.02,0.00,-233,47,-269,-555,296,-64.0,28.3,-63.9,-153.0,36,0.02,0.00,-0.02,192,203,-216,-18,661.0,473.0,A
2,3,carlitos,1323084231,820366,05/12/2011 11:23,no,11,1.42,8.07,-94.4,3,0.00,0.00,-0.02,-20,5,23,-2,600,-305,-128.0,22.5,-161.0,34,0.02,-0.02,-0.02,-289,110,-126,-368,344,513,12.850750,-70.278120,-85.140781,37,0.00,-0.02,0.00,-232,46,-270,-561,298,-63.0,28.3,-63.9,-152.0,36,0.03,-0.02,0.00,196,204,-213,-18,658.0,469.0,A
3,4,carlitos,1323084232,120339,05/12/2011 11:23,no,12,1.48,8.05,-94.4,3,0.02,0.00,-0.03,-22,3,21,-6,604,-310,-128.0,22.1,-161.0,34,0.02,-0.03,0.02,-289,111,-123,-372,344,512,13.431200,-70.393795,-84.873626,37,0.00,-0.02,-0.02,-232,48,-269,-552,303,-60.0,28.1,-63.9,-152.0,36,0.02,-0.02,0.00,189,206,-214,-16,658.0,469.0,A
4,5,carlitos,1323084232,196328,05/12/2011 11:23,no,12,1.48,8.07,-94.4,3,0.02,0.02,-0.02,-21,2,24,-6,600,-302,-128.0,22.1,-161.0,34,0.00,-0.03,0.00,-289,111,-123,-374,337,506,13.378716,-70.428560,-84.853057,37,0.00,-0.02,0.00,-233,48,-270,-554,292,-68.0,28.0,-63.9,-152.0,36,0.02,0.00,-0.02,189,206,-214,-17,655.0,473.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19617,19618,adelmo,1322832937,588376,02/12/2011 13:35,no,864,147.00,-34.80,129.0,21,0.37,-0.02,-0.67,50,26,-193,190,552,-412,-99.4,-33.8,79.0,47,0.55,-0.51,0.25,75,-184,-415,272,-134,-562,36.413180,-22.861971,-113.499833,19,0.32,-0.26,-0.36,-42,66,-168,-618,134,0.0,0.0,0.0,0.0,29,1.73,-1.75,-0.25,-271,-68,-37,-205,-587.0,6.0,E
19618,19619,adelmo,1322832937,596287,02/12/2011 13:35,no,864,145.00,-35.30,130.0,19,0.39,-0.02,-0.67,47,15,-179,192,558,-389,-99.6,-34.5,77.3,45,0.88,-0.71,0.21,52,-163,-406,288,-112,-559,35.152811,-22.971908,-114.525642,18,0.24,-0.24,0.05,-41,62,-164,-618,116,7.0,0.0,0.0,0.0,29,1.59,-1.36,0.00,-271,-91,-43,-151,-635.0,-36.0,E
19619,19620,adelmo,1322832937,636283,02/12/2011 13:35,no,864,145.00,-35.50,130.0,19,0.37,0.00,-0.64,47,13,-177,191,560,-386,-99.6,-35.1,76.3,44,0.98,-0.82,0.23,62,-167,-391,309,-103,-541,30.060284,-20.990184,-120.031804,19,0.22,-0.27,0.21,-38,54,-170,-621,113,-9.0,0.0,0.0,0.0,29,1.54,-1.20,0.05,-263,-99,-45,-116,-654.0,-70.0,E
19620,19621,adelmo,1322832937,964299,02/12/2011 13:35,no,864,143.00,-35.90,131.0,18,0.37,-0.02,-0.59,46,18,-172,190,565,-370,-98.6,-36.7,73.5,41,1.35,-1.00,0.49,70,-164,-359,339,-91,-543,22.863326,-21.756623,-125.245873,19,0.13,-0.14,0.34,-40,42,-176,-628,116,0.0,0.0,0.0,0.0,32,1.48,-0.90,0.05,-270,-141,-5

3). Get rid of the unnecessary columns. HINT: Those columns with "time" in the name and those that are obviously unnecessary.

In [40]:
yourdf = yourdf.drop(['Unnamed: 0' , 'user_name','raw_timestamp_part_1' , 'raw_timestamp_part_2' , 'cvtd_timestamp'] , axis=1)
yourdf

,new_window,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,gyros_belt_x,gyros_belt_y,gyros_belt_z,accel_belt_x,accel_belt_y,accel_belt_z,magnet_belt_x,magnet_belt_y,magnet_belt_z,roll_arm,pitch_arm,yaw_arm,total_accel_arm,gyros_arm_x,gyros_arm_y,gyros_arm_z,accel_arm_x,accel_arm_y,accel_arm_z,magnet_arm_x,magnet_arm_y,magnet_arm_z,roll_dumbbell,pitch_dumbbell,yaw_dumbbell,total_accel_dumbbell,gyros_dumbbell_x,gyros_dumbbell_y,gyros_dumbbell_z,accel_dumbbell_x,accel_dumbbell_y,accel_dumbbell_z,magnet_dumbbell_x,magnet_dumbbell_y,magnet_dumbbell_z,roll_forearm,pitch_forearm,yaw_forearm,total_accel_forearm,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
0,no,11,1.41,8.07,-94.4,3,0.00,0.00,-0.02,-21,4,22,-3,599,-313,-128.0,22.5,-161.0,34,0.00,0.00,-0.02,-288,109,-123,-368,337,516,13.052175,-70.494004,-84.873939,37,0.00,-0.02,0.00,-234,47,-271,-559,293,-65.0,28.4,-63.9,-153.0,36,0.03,0.00,-0.02,192,203,-215,-17,654.0,476.0,A
1,no,11,1.41,8.07,-94.4,3,0.02,0.00,-0.02,-22,4,22,-7,608,-311,-128.0,22.5,-161.0,34,0.02,-0.02,-0.02,-290,110,-125,-369,337,513,13.130740,-70.637505,-84.710647,37,0.00,-0.02,0.00,-233,47,-269,-555,296,-64.0,28.3,-63.9,-153.0,36,0.02,0.00,-0.02,192,203,-216,-18,661.0,473.0,A
2,no,11,1.42,8.07,-94.4,3,0.00,0.00,-0.02,-20,5,23,-2,600,-305,-128.0,22.5,-161.0,34,0.02,-0.02,-0.02,-289,110,-126,-368,344,513,12.850750,-70.278120,-85.140781,37,0.00,-0.02,0.00,-232,46,-270,-561,298,-63.0,28.3,-63.9,-152.0,36,0.03,-0.02,0.00,196,204,-213,-18,658.0,469.0,A
3,no,12,1.48,8.05,-94.4,3,0.02,0.00,-0.03,-22,3,21,-6,604,-310,-128.0,22.1,-161.0,34,0.02,-0.03,0.02,-289,111,-123,-372,344,512,13.431200,-70.393795,-84.873626,37,0.00,-0.02,-0.02,-232,48,-269,-552,303,-60.0,28.1,-63.9,-152.0,36,0.02,-0.02,0.00,189,206,-214,-16,658.0,469.0,A
4,no,12,1.48,8.07,-94.4,3,0.02,0.02,-0.02,-21,2,24,-6,600,-302,-128.0,22.1,-161.0,34,0.00,-0.03,0.00,-289,111,-123,-374,337,506,13.378716,-70.428560,-84.853057,37,0.00,-0.02,0.00,-233,48,-270,-554,292,-68.0,28.0,-63.9,-152.0,36,0.02,0.00,-0.02,189,206,-214,-17,655.0,473.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19617,no,864,147.00,-34.80,129.0,21,0.37,-0.02,-0.67,50,26,-193,190,552,-412,-99.4,-33.8,79.0,47,0.55,-0.51,0.25,75,-184,-415,272,-134,-562,36.413180,-22.861971,-113.499833,19,0.32,-0.26,-0.36,-42,66,-168,-618,134,0.0,0.0,0.0,0.0,29,1.73,-1.75,-0.25,-271,-68,-37,-205,-587.0,6.0,E
19618,no,864,145.00,-35.30,130.0,19,0.39,-0.02,-0.67,47,15,-179,192,558,-389,-99.6,-34.5,77.3,45,0.88,-0.71,0.21,52,-163,-406,288,-112,-559,35.152811,-22.971908,-114.525642,18,0.24,-0.24,0.05,-41,62,-164,-618,116,7.0,0.0,0.0,0.0,29,1.59,-1.36,0.00,-271,-91,-43,-151,-635.0,-36.0,E
19619,no,864,145.00,-35.50,130.0,19,0.37,0.00,-0.64,47,13,-177,191,560,-386,-99.6,-35.1,76.3,44,0.98,-0.82,0.23,62,-167,-391,309,-103,-541,30.060284,-20.990184,-120.031804,19,0.22,-0.27,0.21,-38,54,-170,-621,113,-9.0,0.0,0.0,0.0,29,1.54,-1.20,0.05,-263,-99,-45,-116,-654.0,-70.0,E
19620,no,864,143.00,-35.90,131.0,18,0.37,-0.02,-0.59,46,18,-172,190,565,-370,-98.6,-36.7,73.5,41,1.35,-1.00,0.49,70,-164,-359,339,-91,-543,22.863326,-21.756623,-125.245873,19,0.13,-0.14,0.34,-40,42,-176,-628,116,0.0,0.0,0.0,0.0,32,1.48,-0.90,0.05,-270,-141,-51,-68,-678.0,-98.0,E


4). Label encode the responde variable. HINT: use preprocessing.LabelEncoder(). 

In [41]:
encoder = preprocessing.LabelEncoder()
yourdf['new_window'] = encoder.fit_transform(yourdf['new_window'])
encoder = preprocessing.LabelEncoder()
yourdf['classe'] = encoder.fit_transform(yourdf['classe'])

In [42]:
yourdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19622 entries, 0 to 19621
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   new_window            19622 non-null  int64  
 1   num_window            19622 non-null  int64  
 2   roll_belt             19622 non-null  float64
 3   pitch_belt            19622 non-null  float64
 4   yaw_belt              19622 non-null  float64
 5   total_accel_belt      19622 non-null  int64  
 6   gyros_belt_x          19622 non-null  float64
 7   gyros_belt_y          19622 non-null  float64
 8   gyros_belt_z          19622 non-null  float64
 9   accel_belt_x          19622 non-null  int64  
 10  accel_belt_y          19622 non-null  int64  
 11  accel_belt_z          19622 non-null  int64  
 12  magnet_belt_x         19622 non-null  int64  
 13  magnet_belt_y         19622 non-null  int64  
 14  magnet_belt_z         19622 non-null  int64  
 15  roll_arm           

5). Carry out min-max scaling of the exploratory variables. HINT: use preprocessing.MinMaxScaler().

In [43]:
Y = yourdf['classe']
yourdf = yourdf.drop('classe' , axis=1)

In [44]:
scaler = preprocessing.MinMaxScaler()
df_scaled = scaler.fit_transform(yourdf)

In [45]:
pd.DataFrame(df_scaled)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
0,0.0,0.011587,0.158774,0.550129,0.238440,0.103448,0.319018,0.500000,0.467532,0.482927,0.313305,0.781579,0.091248,0.768025,0.338428,0.144444,0.627750,0.052778,0.507692,0.566726,0.547771,0.431776,0.137931,0.682109,0.552802,0.158126,0.747692,0.862122,0.542753,0.264551,0.215802,0.637931,0.989235,0.038447,0.007452,0.282875,0.468254,0.096626,0.068016,0.919679,0.275910,0.578889,0.052988,0.075000,0.333333,0.848286,0.022074,0.033753,0.707692,0.536977,0.313433,0.647029,0.652357,0.702375
1,0.0,0.011587,0.158774,0.550129,0.238440,0.103448,0.325153,0.500000,0.467532,0.478049,0.313305,0.781579,0.083799,0.796238,0.340611,0.144444,0.627750,0.052778,0.507692,0.568505,0.544586,0.431776,0.135553,0.683706,0.550647,0.157394,0.747692,0.859799,0.543009,0.264071,0.216336,0.637931,0.989235,0.038447,0.007452,0.284404,0.468254,0.099693,0.071255,0.920387,0.277311,0.578611,0.052988,0.075000,0.333333,0.847901,0.022074,0.033753,0.707692,0.536977,0.312076,0.646516,0.655303,0.700921
2,0.0,0.011587,0.158827,0.550129,0.238440,0.103448,0.319018,0.500000,0.467532,0.487805,0.317597,0.784211,0.093110,0.771160,0.347162,0.144444,0.627750,0.052778,0.507692,0.568505,0.544586,0.431776,0.136742,0.683706,0.549569,0.158126,0.754872,0.859799,0.542097,0.265273,0.214929,0.637931,0.989235,0.038447,0.007452,0.285933,0.466270,0.098160,0.066397,0.920860,0.278711,0.578611,0.052988,0.077778,0.333333,0.848286,0.022011,0.033837,0.711795,0.537621,0.316147,0.646516,0.654040,0.698982
3,0.0,0.012746,0.159141,0.549957,0.238440,0.103448,0.325153,0.500000,0.464286,0.478049,0.309013,0.778947,0.085661,0.783699,0.341703,0.144444,0.625494,0.052778,0.507692,0.568505,0.542994,0.439252,0.136742,0.685304,0.552802,0.155198,0.754872,0.859024,0.543987,0.264886,0.215803,0.637931,0.989235,0.038447,0.007389,0.285933,0.470238,0.099693,0.073684,0.922041,0.282913,0.578056,0.052988,0.077778,0.333333,0.847901,0.022011,0.033837,0.704615,0.538907,0.314790,0.647541,0.654040,0.698982
4,0.0,0.012746,0.159141,0.550129,0.238440,0.103448,0.325153,0.515625,0.467532,0.482927,0.304721,0.786842,0.085661,0.771160,0.350437,0.144444,0.625494,0.052778,0.507692,0.566726,0.542994,0.435514,0.136742,0.685304,0.552802,0.153734,0.747692,0.854376,0.543816,0.264770,0.215870,0.637931,0.989235,0.038447,0.007452,0.284404,0.470238,0.098160,0.072065,0.919442,0.271709,0.577778,0.052988,0.077778,0.333333,0.847901,0.022074,0.033753,0.704615,0.538907,0.314790,0.647029,0.652778,0.700921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19617,0.0,1.000000,0.921425,0.180879,0.860724,0.724138,0.432515,0.484375,0.256494,0.829268,0.407725,0.215789,0.450652,0.620690,0.230349,0.223889,0.310209,0.719444,0.707692,0.615658,0.466561,0.482243,0.569560,0.214058,0.238147,0.626647,0.264615,0.027111,0.618783,0.423857,0.122199,0.327586,0.990787,0.034011,0.006325,0.576453,0.505952,0.254601,0.020243,0.882117,0.366947,0.500000,0.446704,0.500000,0.268519,0.913747,0.016571,0.032791,0.232821,0.362701,0.554953,0.550717,0.130051,0.474552
19618,0.0,1.000000,0.910948,0.176572,0.863510,0.655172,0.438650,0.484375,0.256494,0.814634,0.360515,0.252632,0.454376,0.639498,0.255459,0.223333,0.306261,0.714722,0.676923,0.645018,0.434713,0.474766,0.542212,0.247604,0.247845,0.638360,0.287179,0.029435,0.614681,0.423490,0.118845,0.310345,0.990399,0.034381,0.007608,0.577982,0.498016,0.260736,0.020243,0.877864,0.376751,0.500000,0.446704,0.500000,0.268519,0.908356,0.017798,0.033837,0.232821,0.347910,0.546811,0.578381,0.109848,0.454193
19619,0.0,1.000000,0.910948,0.174849,0.863510,0.655172,0.432515,0.500000,0.266234,0.814634,0.351931,0.257895,0.452514,0.645768,0.258734,0.223333,0.302876,0.711944,0.661538,0.653915,0.417197,0.478505,0.554102,0.241214,0.264009,0.653734,0.2

6). Choose an algorithm and carry out the predictive analysis.

- Optimize the hyperparameter(s)
- Calculate the accuracy.
- Aim for upper 90% accuracy.

In [46]:
x_train, x_test, y_train, y_test = train_test_split(   df_scaled, Y, test_size=0.33, random_state=42)

In [47]:
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
adb = AdaBoostClassifier()
gb = GradientBoostingClassifier()
xgb = XGBClassifier()

In [48]:
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
print("KNN Accuracy : ",metrics.accuracy_score(y_test,y_pred))

KNN Accuracy :  0.9478072884496603


In [49]:
dt.fit(x_train,y_train)
y_pred = dt.predict(x_test)
print("DecisionTree Accuracy : ",metrics.accuracy_score(y_test,y_pred))

DecisionTree Accuracy :  0.980697961704756


In [50]:
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
print("RandomForest Accuracy : ",metrics.accuracy_score(y_test,y_pred))

RandomForest Accuracy :  0.9983014206300185


In [51]:
adb.fit(x_train,y_train)
y_pred = adb.predict(x_test)
print("AdaBoost Accuracy : ",metrics.accuracy_score(y_test,y_pred))

AdaBoost Accuracy :  0.7251389746757257


In [52]:
gb.fit(x_train,y_train)
y_pred = gb.predict(x_test)
print("GradientBoosting Accuracy : ",metrics.accuracy_score(y_test,y_pred))

GradientBoosting Accuracy :  0.9905806053119209


In [53]:
xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)
print("XGB Accuracy : ",metrics.accuracy_score(y_test,y_pred))

XGB Accuracy :  0.9759110562075355
